In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch.nn as nn
import torch
import pytorch_lightning as pl
from panoradio_hf.data import IQDataModel

In [2]:
dm = IQDataModel()
dm.setup(stage="")
dl = dm.train_dataloader()
batch_data, modeid, snrid = dl.__iter__().__next__()
batch_data = batch_data[:10,:,:]

In [8]:
from panoradio_hf.network import IQDataEmbedder, FrequentistClassifier

In [9]:
embed = IQDataEmbedder()
clf = FrequentistClassifier()

AttributeError: cannot assign module before Module.__init__() call

In [6]:
lo = embed(batch_data)

In [37]:
nn.Flatten(1)(layer_out).shape

torch.Size([10, 8192])

In [11]:
help(nn.LSTM)

Help on class LSTM in module torch.nn.modules.rnn:

class LSTM(RNNBase)
 |  LSTM(*args, **kwargs)
 |
 |  __init__(input_size,hidden_size,num_layers=1,bias=True,batch_first=False,dropout=0.0,bidirectional=False,proj_size=0,device=None,dtype=None)
 |
 |  Apply a multi-layer long short-term memory (LSTM) RNN to an input sequence.
 |  For each element in the input sequence, each layer computes the following
 |  function:
 |
 |  .. math::
 |      \begin{array}{ll} \\
 |          i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi}) \\
 |          f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf}) \\
 |          g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg}) \\
 |          o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho}) \\
 |          c_t = f_t \odot c_{t-1} + i_t \odot g_t \\
 |          h_t = o_t \odot \tanh(c_t) \\
 |      \end{array}
 |
 |  where :math:`h_t` is the hidden state at time `t`, :math:`c_t` is the cell
 |  state at time `t`, :math:`x_t` 